# HDMI on PYNQ - Embedded Operating System
----------
## MP4 to HDMI
Zonder filter. change 'noFilter' into '1' for filter *(currently not working, errors in hardware design)*.

In [4]:
from pynq import Overlay, allocate, MMIO
import cv2
import numpy as np
import time

# ### Stap 1: Laad de Overlay (zonder filter)
# Laad de bitstream en hardwarebeschrijving.
bit_path = "/home/xilinx/jupyter_notebooks/base/video/data/design_1_wrapper_noFilter.bit"
hwh_path = "/home/xilinx/jupyter_notebooks/base/video/data/design_1_wrapper_noFilter.hwh"
overlay = Overlay(bit_path)
overlay.download()
print("Bitstream zonder filter geladen.")

# Controleer beschikbare IP's
print("Beschikbare IP's in de overlay:")
print(overlay.ip_dict.keys())

# ### Stap 2: Framebuffer toewijzen
# Reserveer een framebuffer in het DDR-geheugen.
hdmi_width, hdmi_height, hdmi_bpp = 1920, 1080, 24  # Resolutie en bits per pixel
frame_buffer = allocate(shape=(hdmi_height, hdmi_width, 3), dtype=np.uint8)
print(f"Framebuffer fysiek adres: {hex(frame_buffer.physical_address)}")

# ### Stap 3: Directe toegang tot de VDMA via MMIO
# Haal de VDMA basisadres op (vervang "axi_vdma_0" indien nodig).
vdma_details = overlay.ip_dict["axi_vdma_0"]
vdma_base_addr = vdma_details["phys_addr"]
vdma_addr_range = 0xFFFF  # Typisch bereik van VDMA registers

# Maak een MMIO-object voor directe toegang tot de VDMA
vdma_mmio = MMIO(vdma_base_addr, vdma_addr_range)
print(f"VDMA basisadres: {hex(vdma_base_addr)}")

# ### Stap 4: Configureer de VDMA
# Reset en configureer de VDMA
print("VDMA configuratie gestart...")

# Soft reset VDMA
vdma_mmio.write(0x00, 0x00000004)  # Soft reset MM2S
print("VDMA soft reset uitgevoerd...")
time.sleep(0.1)  # Wacht op reset

# Lees de status na de reset
reset_status = vdma_mmio.read(0x04)
print(f"Status na reset: 0x{reset_status:X}")

# Start de VDMA opnieuw
vdma_mmio.write(0x00, 0x00000001)  # MM2S Control Register: Zet de VDMA aan

# Stel de framebuffer-adres in
vdma_mmio.write(0x5C, frame_buffer.physical_address)  # Eerste framebuffer

# Test hardgecodeerde waarden voor horizontale grootte en stride
horizontal_size = hdmi_width * (hdmi_bpp // 8)
vdma_mmio.write(0x54, horizontal_size)  # MM2S Horizontal Size (bytes)
vdma_mmio.write(0x58, horizontal_size)  # MM2S Frame Delay and Stride (bytes)

# Stel de verticale grootte in
vdma_mmio.write(0x50, hdmi_height)  # MM2S Vertical Size (lijnen)

# Forceer VDMA-configuratie en controleer registers
print("Forceer VDMA-configuratie...")
time.sleep(0.05)  # Korte pauze
horizontal_size_set = vdma_mmio.read(0x54)
stride_set = vdma_mmio.read(0x58)
print(f"Verwachte horizontale grootte: {horizontal_size}, Gelezen: {horizontal_size_set}")
print(f"Verwachte stride: {horizontal_size}, Gelezen: {stride_set}")

# Verticale grootte
time.sleep(0.05)
vertical_size_set = vdma_mmio.read(0x50)
print(f"Verwachte verticale grootte: {hdmi_height}, Gelezen: {vertical_size_set}")

# Framebuffer-adres
time.sleep(0.05)
framebuffer_addr_set = vdma_mmio.read(0x5C)
print(f"Verwacht framebuffer-adres: {hex(frame_buffer.physical_address)}, Gelezen: {hex(framebuffer_addr_set)}")

# Controleer opnieuw de status
status = vdma_mmio.read(0x04)  # MM2S Status Register
print(f"MM2S Status Register na configuratie: 0x{status:X}")

# Controleer of de waarden correct zijn ingesteld
if horizontal_size_set != horizontal_size or vertical_size_set != hdmi_height or stride_set != horizontal_size:
    print("Waarden in VDMA registers blijven niet overeenkomen. Mogelijk hardwareprobleem of verkeerde adressering.")
else:
    print("VDMA configuratie correct toegepast.")

# Controleer de statusbits
if status & 0x1:
    print("VDMA is gestopt (Halted).")
else:
    print("VDMA draait correct.")

if status & 0x10:
    print("VDMA interne fout gedetecteerd! Controleer klok- of synchronisatieproblemen.")
if status & 0x20:
    print("VDMA slave fout gedetecteerd! Controleer AXI-adressen en geheugenmapping.")
if status & 0x40:
    print("VDMA decodeer fout gedetecteerd! Mogelijk een ongeldige configuratie.")

# ### Stap 5: Stream Video Frames naar de VDMA
# Gebruik OpenCV om frames van een MP4-bestand naar de framebuffer te sturen.
video_path = "/home/xilinx/jupyter_notebooks/common/data/Surfing.mp4"
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    raise IOError(f"Kan video niet openen: {video_path}")

print("Video geopend. Streaming gestart...")

try:
    frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Einde van de video.")
            break

        # Schaal frame naar HDMI-resolutie
        frame_resized = cv2.resize(frame, (hdmi_width, hdmi_height))

        # Kopieer frame naar framebuffer
        np.copyto(frame_buffer, frame_resized)

        frame_count += 1
        if frame_count % 30 == 0:  # Print alleen elke 30 frames
            print(f"{frame_count} frames verzonden.")

        # Debug: Controleer VDMA status tijdens streaming
        if frame_count % 60 == 0:
            status = vdma_mmio.read(0x04)
            print(f"VDMA Status tijdens streaming: 0x{status:X}")

except KeyboardInterrupt:
    print("Streaming onderbroken door gebruiker.")

finally:
    cap.release()
    print("Streaming gestopt.")


Bitstream zonder filter geladen.
Beschikbare IP's in de overlay:
dict_keys(['axi_vdma_0', 'processing_system7_0'])
Framebuffer fysiek adres: 0x16f00000
VDMA basisadres: 0x43000000
VDMA configuratie gestart...
VDMA soft reset uitgevoerd...
Status na reset: 0x10001
Forceer VDMA-configuratie...
Verwachte horizontale grootte: 5760, Gelezen: 5760
Verwachte stride: 5760, Gelezen: 5760
Verwachte verticale grootte: 1080, Gelezen: 1080
Verwacht framebuffer-adres: 0x16f00000, Gelezen: 0x16f00000
MM2S Status Register na configuratie: 0x11000
VDMA configuratie correct toegepast.
VDMA draait correct.
Video geopend. Streaming gestart...
30 frames verzonden.
60 frames verzonden.
VDMA Status tijdens streaming: 0x11000
90 frames verzonden.
120 frames verzonden.
VDMA Status tijdens streaming: 0x11000
150 frames verzonden.
180 frames verzonden.
VDMA Status tijdens streaming: 0x11000
210 frames verzonden.
240 frames verzonden.
VDMA Status tijdens streaming: 0x11000
270 frames verzonden.
Einde van de video

## RTSP to HDMI
Zonder filter. change 'noFilter' into '1' for filter *(currently not working, errors in hardware design)*.

In [ ]:
from pynq import Overlay, allocate, MMIO
import cv2
import numpy as np
import time

# ### Stap 1: Laad de Overlay (zonder filter)
bit_path = "/home/xilinx/jupyter_notebooks/base/video/data/design_1_wrapper_noFilter.bit"
hwh_path = "/home/xilinx/jupyter_notebooks/base/video/data/design_1_wrapper_noFilter.hwh"
overlay = Overlay(bit_path)
overlay.download()
print("Bitstream zonder filter geladen.")

# Controleer beschikbare IP's
print("Beschikbare IP's in de overlay:")
print(overlay.ip_dict.keys())

# ### Stap 2: Framebuffer toewijzen
hdmi_width, hdmi_height, hdmi_bpp = 1920, 1080, 24  # Resolutie en bits per pixel
frame_buffer = allocate(shape=(hdmi_height, hdmi_width, 3), dtype=np.uint8)
print(f"Framebuffer fysiek adres: {hex(frame_buffer.physical_address)}")

# ### Stap 3: Directe toegang tot de VDMA via MMIO
vdma_details = overlay.ip_dict["axi_vdma_0"]
vdma_base_addr = vdma_details["phys_addr"]
vdma_addr_range = 0xFFFF
vdma_mmio = MMIO(vdma_base_addr, vdma_addr_range)
print(f"VDMA basisadres: {hex(vdma_base_addr)}")

# ### Stap 4: Configureer de VDMA
print("VDMA configuratie gestart...")
vdma_mmio.write(0x00, 0x00000004)  # Soft reset MM2S
print("VDMA soft reset uitgevoerd...")
time.sleep(0.1)

reset_status = vdma_mmio.read(0x04)
print(f"Status na reset: 0x{reset_status:X}")

vdma_mmio.write(0x00, 0x00000001)  # MM2S Control Register: Zet de VDMA aan
vdma_mmio.write(0x5C, frame_buffer.physical_address)  # Eerste framebuffer

horizontal_size = hdmi_width * (hdmi_bpp // 8)
vdma_mmio.write(0x54, horizontal_size)  # MM2S Horizontal Size (bytes)
vdma_mmio.write(0x58, horizontal_size)  # MM2S Frame Delay and Stride (bytes)
vdma_mmio.write(0x50, hdmi_height)  # MM2S Vertical Size (lijnen)

time.sleep(0.05)
horizontal_size_set = vdma_mmio.read(0x54)
stride_set = vdma_mmio.read(0x58)
vertical_size_set = vdma_mmio.read(0x50)
framebuffer_addr_set = vdma_mmio.read(0x5C)

print(f"Verwachte horizontale grootte: {horizontal_size}, Gelezen: {horizontal_size_set}")
print(f"Verwachte stride: {horizontal_size}, Gelezen: {stride_set}")
print(f"Verwachte verticale grootte: {hdmi_height}, Gelezen: {vertical_size_set}")
print(f"Verwacht framebuffer-adres: {hex(frame_buffer.physical_address)}, Gelezen: {hex(framebuffer_addr_set)}")

status = vdma_mmio.read(0x04)
print(f"MM2S Status Register na configuratie: 0x{status:X}")

if horizontal_size_set != horizontal_size or vertical_size_set != hdmi_height or stride_set != horizontal_size:
    print("Waarden in VDMA registers blijven niet overeenkomen. Mogelijk hardwareprobleem of verkeerde adressering.")
else:
    print("VDMA configuratie correct toegepast.")

if status & 0x1:
    print("VDMA is gestopt (Halted).")
else:
    print("VDMA draait correct.")

# ### Stap 5: Stream RTSP Frames naar de VDMA
rtsp_url = "rtsp://192.168.0.205:8554/test"  #TEST URL, aanpassen naar EdgeCam
cap = cv2.VideoCapture(rtsp_url)

if not cap.isOpened():
    raise IOError(f"Kan RTSP-stream niet openen: {rtsp_url}")

print("RTSP-stream geopend. Streaming gestart...")

try:
    frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Einde van de stream of fout bij het lezen.")
            break

        # Schaal frame naar HDMI-resolutie
        frame_resized = cv2.resize(frame, (hdmi_width, hdmi_height))

        # Kopieer frame naar framebuffer
        np.copyto(frame_buffer, frame_resized)

        frame_count += 1
        if frame_count % 30 == 0:  # Print alleen elke 30 frames
            print(f"{frame_count} frames verzonden.")

        # Debug: Controleer VDMA status tijdens streaming
        if frame_count % 60 == 0:
            status = vdma_mmio.read(0x04)
            print(f"VDMA Status tijdens streaming: 0x{status:X}")

except KeyboardInterrupt:
    print("Streaming onderbroken door gebruiker.")

finally:
    cap.release()
    print("Streaming gestopt.")
